In [741]:
import cv2 as cv
import numpy as np
import platform

In [742]:
# list to store previous frames, needed to compute temporal changes
# using it as a buffer here, with length = prevFrameNumber
frameList = []

fps =60
# number of previous frames to process:
prevFrameNumber = int(fps * 3)
b_FrameNumber = int(prevFrameNumber * (1 / 3))
g_FrameNumber = int(prevFrameNumber * (2 / 3))
r_FrameNumber = int(prevFrameNumber * (3 / 3))

currentFrameNumber = 0

# treshold for the final output:
threshold = 150

# detects os of user:
userPlatform = platform.system()

# selecting the videocapture backend in Linux is necessary in some cases
# change 0 to whatever webcam id is required
if userPlatform == "Linux":
    cam = (0, cv.CAP_V4L2)
elif userPlatform in ["Windows", "Darwin"]:
    cam = (0,)

Posy method: 
1) Duplicate video
2) Invert video, and decrease opacity to about 50%
3) "Shift the time of the video"?

<!-- trying a simple differnce between frames for now -->

In [743]:
# create a video capture object,
# sourcing it from a webcam with id 0 (the default one)
# replace the 0 with a filepath if using a video file instead
cap = cv.VideoCapture("./../windmill.mp4")
# cap = cv.VideoCapture(*cam)

# set video fps to 60, because why not?
cap.set(cv.CAP_PROP_FPS, fps)


# nPrevFram is the number of frames in the past that this function will look
# Assume that frameList has that many frames already in BRG mode
# Also current frames is assumed to last frame in frameList
# blend is number for 0-1: 0 for completely show change 1 to only show original image
def greyMotionExtraction(frameList, nPrevFrame, blend):
    currentFrame = frameList[-1]
    currentFrame = cv.cvtColor(currentFrame, cv.COLOR_BGR2GRAY)
    previousFrame = frameList[-nPrevFrame]
    previousFrame = cv.cvtColor(previousFrame, cv.COLOR_BGR2GRAY)
    previousFrame = cv.bitwise_not(previousFrame)
    # # previousFrame = cv.cvtColor(previousFrame, cv.COLOR_BGR2GRAY)
    # proc_frame = cv.addWeighted(currentFrame, blend, previousFrame, 1 - blend, 0)
    # _, mask = cv.threshold(proc_frame, 140, 255, cv.THRESH_BINARY)
    # height, width = mask.shape
    # solid_color_frame = np.full((height, width, 3), (255, 0, 0), dtype=np.uint8)

    # mask = cv.bitwise_and(solid_color_frame, solid_color_frame, mask=mask)

    proc_frame = cv.addWeighted(currentFrame, blend, previousFrame, 1 - blend, 0)
    proc_frame = cv.cvtColor(proc_frame, cv.COLOR_GRAY2BGR)
    return proc_frame


def colourMotionExtraction(frameList, nBPrevFrame, nGPrevFrame, nRPrevFrame, blend):
    currentFrame = frameList[-1]
    r_Frame = frameList[-r_FrameNumber].copy()
    r_Frame[:, :, 0] = 0
    r_Frame[:, :, 1] = 0

    g_Frame = frameList[-g_FrameNumber].copy()
    g_Frame[:, :, 0] = 0
    g_Frame[:, :, 2] = 0

    b_Frame = frameList[-b_FrameNumber].copy()
    b_Frame[:, :, 1] = 0
    b_Frame[:, :, 2] = 0
    previousFrame = currentFrame.copy()  # To get the correct shape
    previousFrame[:, :, 0] = b_Frame[:, :, 0]
    previousFrame[:, :, 1] = g_Frame[:, :, 1]
    previousFrame[:, :, 2] = r_Frame[:, :, 2]

    previousFrame = cv.bitwise_not(previousFrame)
    proc_frame = cv.addWeighted(currentFrame, blend, previousFrame, 1 - blend, 0)
    return proc_frame


while True:

    # capture each frame from the video feed -----------------------------------------------------------------------------------------------------------
    _, frame = cap.read()

    if frame is None:
        break

    old = frame

    #  process each frame here:   ----------------------------------------------------------------------------------------------------------------------

    frame = cv.GaussianBlur(frame, (5, 5), 0)
    currentFrameNumber += 1

    frameList.append(frame)

    if currentFrameNumber >= prevFrameNumber:
        # currentFrameNumber=0
        frame = greyMotionExtraction(frameList, prevFrameNumber, 0.5)
        # frame = colourMotionExtraction(
        #     frameList, b_FrameNumber, g_FrameNumber, r_FrameNumber, 0.4
        # )
        frameList.pop(0)
        # use the treshold for using it as some sort of special effect by addign it back to the original
        # _, frame = cv.threshold(frame, treshold, 255, cv.THRESH_BINARY)
        # frame = cv.cvtColor(frame, cv.COLOR_GRAY2BGR)
        # display the video in a window, per frame:
        hori_compare = np.concatenate((old, frame), axis=1)
        cv.namedWindow("video", cv.WINDOW_NORMAL)
        cv.imshow("video", hori_compare)

    # quit the loop if 'q' is pressed
    #! close the window by typing "q", closing it normally will cause the kernel to crash,
    #! among other things

    if cv.waitKey(1) == ord("q"):
        break

cap.release()
cv.destroyAllWindows()